In [33]:
# This script is for processing Safegraph Data, only 1 input, output tidied up place visitation data with the number of visitors.
# Take "Original Safegraph Data for Dataset 3 (Whole USA) ———— Fitness center" for example

In [34]:
# import packages
import pandas as pd
import numpy as np
import re
import geopandas as gpd
import os
from scipy.optimize import curve_fit

In [35]:
# process pattern dataset
def pattern_process(df, name, filenames1, folder_name):
    print("Start pattern process " + name)
    # delete NA
    df_tract_visitor = df['visitor_home_aggregation']
    df_tract_visitor.rename("visitors", inplace = True)
    df_tract_visitor.dropna(inplace = True)
    # delete {}
    df_tract_visitor1 = pd.DataFrame(df_tract_visitor)
    df_tract_visitor2 = df_tract_visitor1[~df_tract_visitor1['visitors'].isin(['{}'])]
    # fix format error, like 32432432“”：4
    df_tract_visitor3 = df_tract_visitor2[df_tract_visitor2["visitors"].str.contains('""')]
    df_tract_visitor3.reset_index(drop = True, inplace = True)
    for i in range(len(df_tract_visitor3)):
        df_tract_visitor3["visitors"][i] = "{" + '"' + df_tract_visitor3["visitors"][i][0:11] + '"' + ":" + df_tract_visitor3["visitors"][i].split(":")[1] + "}"
    df_tract_visitor4 = df_tract_visitor2[~df_tract_visitor2["visitors"].str.contains('""')]
    df_tract_visitor5 = pd.concat([df_tract_visitor3, df_tract_visitor4])
    df_tract_visitor5.reset_index(drop = True, inplace = True)
    # test whether or not there will be same census tract, the answer is not, but I just keep it
    for p in range(len(df_tract_visitor5)):
        test_same_value = df_tract_visitor5["visitors"][p] 
        test_same_value1 = re.findall(r'"([^"]*)"', test_same_value)
        set_lst = set(test_same_value1)
        if len(set_lst) != len(test_same_value1):
            print(name + " ———> " + "pattern: " + 'Same value, wrong!!!')
    # dict to dataframe
    list_data = df_tract_visitor5['visitors'].values.tolist()
    dfss = []
    for x in range(len(list_data)):
        data_visitor = pd.DataFrame.from_dict(eval(list_data[x]), orient = 'index', columns = ['values'])
        dfss.append(data_visitor)
    data_visitor_f = pd.concat(dfss)
    global data_visitor1
    data_visitor1 = data_visitor_f.reset_index()
    data_visitor1.rename(columns={'index':'tractcode','values':'visitors'},inplace=True) 
    # test whether or not they are all census tract, with the 11 number of digits
    for y in range(len(data_visitor1)):
        if len(data_visitor1['tractcode'][y]) != 11:
            print(name + " ———> " + "pattern: " + str(y) + "is not tract!!!")
    # data_visitor1.to_csv('{}/{}pattern{}.csv'.format(filenames1, folder_name, filenames1.split("2018")[1].split("-2022-")[0]),encoding='utf_8_sig',index=False)
    return data_visitor1

In [36]:
# curve estimation
def curve_estimation(dfc, filename_input, folder_name):
    print("begin curve estimation!")
    df1 = dfc.groupby("visitors").count()
    df1.reset_index(inplace = True)
    df2 = df1[df1["visitors"] != 4]
    df2.rename(columns={'vsitors':'vsitors','tractcode':'counts'},inplace=True) 
    
    xdata = np.array(df2["visitors"])
    ydata = np.array(df2["counts"])
    
    def func(x,a,b):
        return a*x**b
    
    popt, pcov = curve_fit(func, xdata, ydata)
    residuals = ydata - func(xdata, *popt)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)
    fol = {"a" : popt[0],
       "b" : popt[1],
       "R squared" : r_squared}
    fol1 = pd.DataFrame([fol])
    fol1.to_csv('{}/{}curve_estimation_formula.csv'.format(filename_input, folder_name),encoding='utf_8_sig',index=False)
    num_2 = func(2, *popt)    
    num_3 = func(3, *popt)  
    num_4 = func(4, *popt)  
    sum_all = num_2 + num_3 + num_4
    num_2_re = round(df1["tractcode"][0] * (num_2/sum_all))
    num_3_re = round(df1["tractcode"][0] * (num_3/sum_all))
    df_4_1 = dfc[dfc["visitors"] == 4]
    # Shuffle the dataframe order
    from sklearn.utils import shuffle
    df_4 = shuffle(df_4_1, random_state = 5) # set random seed to make everytime the same
    df_4.iloc[:num_2_re, 1] = 2 # must use iloc, otherwise, will come up to warning
    df_4.iloc[num_2_re : (num_2_re+num_3_re), 1] = 3 # must use iloc, otherwise, will come up to warning
    df_no_4 = dfc[dfc["visitors"] != 4]
    dfs_1 = [df_4,df_no_4]
    df_final = pd.concat(dfs_1)
    # df_final.to_csv('C:/Users/ryanz/Desktop/210717-Obesity Estimation/04 Data Sample_NY/03 Safegraph Data/02 NY_Pattern_Natrual Park_712190_3/712190_NY_Natrual Park_data_curve_estimate_last_1.csv',encoding='utf_8_sig',index=False) # output
    df_final_1 = df_final.groupby(["tractcode"])["visitors"].sum().reset_index()
    global df_final_2 
    df_final_2 = df_final_1[~df_final_1['tractcode'].str.contains('CA')]
    # df_final_2.to_csv('{}/{}curve_estimation_pattern_final.csv'.format(filename_input, folder_name),encoding='utf_8_sig',index=False)
    return df_final_2

In [37]:
# process summary dataset
def summary_process(data, name, filenames1, folder_name):
    print("Start summary process " + name)
    # read home_panel_summary data
    data[["census_block_group"]] = data[["census_block_group"]].astype(str)
    data = data[['census_block_group','number_devices_residing']]
    data1 = data[~data['census_block_group'].str.contains('CA')].reset_index()
    # some census tracts miss 0, add 0
    for n in range(len(data1)):
        if len(data1['census_block_group'][n]) != 12:
            data1['census_block_group'][n] = "0" + data1['census_block_group'][n]
    # test whether or not they are all sensus block groups, with the 12 number of digits
    m = 0
    for m in range(len(data1)):
        if len(data1['census_block_group'][m]) != 12:
            print(name + " ———> " + "home_panel_summary: " + str(m) + "is not tract!!!")
    # groupby
    data2 = data1.groupby(["census_block_group"])["number_devices_residing"].sum().reset_index()
    data2 = data2[["census_block_group","number_devices_residing"]]
    # delete last digit, change it to census tract
    data2['census_block_group'] = data2['census_block_group'].str[:-1] # https://segmentfault.com/q/1010000023190974, this can save a hug of time!
    global data3
    data3 = data2.groupby(["census_block_group"])["number_devices_residing"].sum().reset_index()
    # print(name + " ———> " + "pattern and home_panel_summary: " + "completed!")
    # data3.to_csv('{}/{}summary{}.csv'.format(filenames1, folder_name, filenames1.split("2018")[1].split("-2022-")[0]),encoding='utf_8_sig',index=False)
    return data3

In [39]:
# main 
if __name__ == '__main__':
    
    # this is the only input
    filename_input = '../02 Dataset/01 Original Safegraph Data for Dataset 3 (Whole USA)/01 Pattern_Fitness and Recreational Sports centers_713940' # only input
    
    dir = filename_input
    folder_name = filename_input[-6:] + "_" + filename_input.split("Pattern_")[1].split("_")[0] + "_"
    filenames_1 = os.listdir(dir) 
    filenames = []
    for h in range(len(filenames_1)):
        filenames.append(filenames_1[h])
        
    dfs_p = []
    dfs_s = []
    index = 1
    
    for name in filenames:
        print(str(index) +" document processing")
        filenames1 = os.path.join(dir,name)
        filenames2 = os.path.join(filenames1,"2022-01-28-23-2018-0" + str(index) + "-core_poi-patterns.csv") 
        filenames3 = os.path.join(filenames1,"2022-01-28-23-2018-0" + str(index) + "-home_panel_summary.csv") 
        
        df = pd.read_csv(filenames2)
        pattern_process(df, name, filenames1, folder_name)
        dfs_p.append(data_visitor1)
        
        data = pd.read_csv(filenames3, converters = {u'census_block_group':str})
        summary_process(data, name, filenames1, folder_name)
        dfs_s.append(data3)

        index += 1
        
    pattern_final = pd.concat(dfs_p)
    # pattern_final.to_csv('{}/{}pattern_final.csv'.format(filename_input, folder_name),encoding='utf_8_sig',index=False)
    curve_estimation(pattern_final, filename_input, folder_name)
    
    summary_final = pd.concat(dfs_s)
    summary_final_1 = summary_final.groupby(["census_block_group"])["number_devices_residing"].sum().reset_index()
    # summary_final_1.to_csv('{}/{}summary_final.csv'.format(filename_input, folder_name),encoding='utf_8_sig',index=False)
    
    pattern_summary_final = pd.merge(df_final_2, summary_final_1, left_on = "tractcode", right_on = "census_block_group", how = "left")
    pattern_summary_final1 = pattern_summary_final[['tractcode','visitors','number_devices_residing']]
    pattern_summary_final1.rename(columns={'tractcode':'tractfps','visitors':'visitors','number_devices_residing':'devices'},inplace=True)
    pattern_summary_final1["ratio"] = pattern_summary_final1['visitors']/pattern_summary_final1['devices']
    pattern_summary_final1.replace(np.nan, 0, inplace = True)
    pattern_summary_final1.replace(np.inf, 0, inplace = True)
    pattern_summary_final1[["tractfps"]] = pattern_summary_final1[["tractfps"]].astype(str)
    pattern_summary_final1[["visitors"]] = pattern_summary_final1[["visitors"]].astype(int)
    pattern_summary_final1[["devices"]] = pattern_summary_final1[["devices"]].astype(int)
    pattern_summary_final1[["ratio"]] = pattern_summary_final1[["ratio"]].astype(float)
    # pattern_summary_final1.to_csv('{}/{}pattern_summary_last.csv'.format(filename_input, folder_name),encoding='utf_8_sig',index=False)
    
    # Use geopandas to connect
    tract = gpd.read_file('../Data/02 Census Tract Shpfiles of Three Cities/CDC data_Tract_Ob_last.shp')
    tract_final = pd.merge(tract, pattern_summary_final1, left_on = "GEOID", right_on = "tractfps", how = "left")
    tract_final_1 = tract_final[['GEOID','INTPTLAT','INTPTLON','geometry', 'visitors', 'devices','ratio','obesity_cr','obesity__2','pop']]
    tract_final_1.to_file('{}/{}.shp'.format(filename_input, folder_name[:-1]),encoding='utf-8')
    tract_final_3 = pd.DataFrame(tract_final_1)
    tract_final_4 = tract_final_3[['GEOID','INTPTLAT','INTPTLON', 'visitors', 'devices','ratio','obesity_cr','obesity__2','pop']]
    tract_final_4.to_csv('{}/{}.csv'.format(filename_input, folder_name[:-1]),encoding='utf_8_sig',index=False)
    print(folder_name + "completed")

1 document processing
Start pattern process 713940-CORE_POI-PATTERNS-2018_01-2022-01-28
Start summary process 713940-CORE_POI-PATTERNS-2018_01-2022-01-28
2 document processing
Start pattern process 713940-CORE_POI-PATTERNS-2018_02-2022-01-28
Start summary process 713940-CORE_POI-PATTERNS-2018_02-2022-01-28
3 document processing
Start pattern process 713940-CORE_POI-PATTERNS-2018_03-2022-01-28
Start summary process 713940-CORE_POI-PATTERNS-2018_03-2022-01-28
4 document processing
Start pattern process 713940-CORE_POI-PATTERNS-2018_04-2022-01-28
Start summary process 713940-CORE_POI-PATTERNS-2018_04-2022-01-28
5 document processing
Start pattern process 713940-CORE_POI-PATTERNS-2018_05-2022-01-28
Start summary process 713940-CORE_POI-PATTERNS-2018_05-2022-01-28
6 document processing
Start pattern process 713940-CORE_POI-PATTERNS-2018_06-2022-01-28
Start summary process 713940-CORE_POI-PATTERNS-2018_06-2022-01-28
7 document processing
Start pattern process 713940-CORE_POI-PATTERNS-2018_07